# Análise das Stories

Notebook para o auxílio da análise das stories do chatbot.

### Configurando e Imports

In [1]:
import rasa_nlu
import rasa_core
from IPython.display import IFrame

print("Versões utilizadas neste jupyter notebook:")
print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))


/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Versões utilizadas neste jupyter notebook:
rasa_nlu: 0.15.0 rasa_core: 0.14.1


## Análise e Avaliação das Stories

### Visualizaçõa do Fluxo de Conversa

O comando abaixo monta um grafo com a relação das `intents` e `utters` do chatbot, ou seja, você vai conseguir visualizar o fluxo de conversa do seu chatbot.

Esta visualização é importante para verificar possíveis problemas na estrutura do seu chatbot e se ele realmente chega em determinadas "**falas**" da conversa.

In [2]:
!python -m rasa_core.visualize -d $BOT_DOMAIN_PATH -s $BOT_STORIES_PATH -o chat_graph.html

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_core.visualize' found in sys.modules after import of package 'rasa_core', but prior to execution of 'rasa_core.visualize'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-05-03 21:20:43 INFO     apscheduler.scheduler  - Scheduler started

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

2019-05-03 21:20:44 INFO     __main__  - Starting to visualize stories...
Processed Story Blocks: 100%|███| 34/34 [00:00<00:00, 5308.85it/s, # trackers=1]
2019-05-03 21:20:44 INFO     __main__  - Finished graph creation. Saved into file:///work/n

O Rasa gerou uma página `html` com o grafo de conversa, para facilitar, você pode visualizar o grafo no arquivo `chat_graph.html` aqui na próxima celula.

Use o mouse para dar zoom e arrastar sobre o conteúdo do grafo. Altere `width` e `height` se desejar.

* Dica: caso você se perca com **zoom in** ou **zoom out** no grafo, apeas re-execute a célula abaixo e ele irá reaparecer na célula.

In [3]:
IFrame(src='./chat_graph.html', width=900, height=700)

### Avaliação das Stories

Outra forma de analisar seu chatbot é por meio da própria avaliação do Rasa, ele gera uma matriz de confuzão com os dados fornecidos nas `stories` e do resultado do treinamento armazenado na pasta `models`.

* Caso você ainda não tenha treinado seu chatbot execute a céclula abaixo para treina-lo.

In [4]:
!make train -C $BOT_DIR_PATH

make: Entering directory '/work/bot'
python3 -m rasa_nlu.train -c nlu_config.yml --fixed_model_name current \
  --data data/intents/ -o models --project nlu --verbose
/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.train' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.train'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-05-03 21:20:47 INFO     rasa_nlu.training_data.loading  - Training data format of data/intents/aleatorio.md is md
2019-05-03 21:20:47 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 197 (24 distinct intents)
	- Found intents: 'religiao', 'linguagens', 'license', 'como_estou', 'filme', 'hobby', 'de_onde_voce_eh', 'signo', 'filhos', 'relationship', '

2019-05-03 21:21:02 INFO     validator  - Domain verified
2019-05-03 21:21:02 ERROR    validator  - The utter action_test is used in the stories story file data/stories/actions.md (line: 5) but it's not a valid utter.
2019-05-03 21:21:04 INFO     apscheduler.scheduler  - Scheduler started
2019-05-03 21:21:04 DEBUG    apscheduler.scheduler  - Looking for jobs to run
2019-05-03 21:21:04 DEBUG    apscheduler.scheduler  - No jobs; waiting until a job is added

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

2019-05-03 21:21:05 DEBUG    rasa_core.training.generator  - Number of augmentation rounds is 3
2019-05-03 21:21:05 DEBUG    rasa_core.training.generator  - Starting data generation round 0 ... (with 1 trackers)
Processed Story Blocks: 100%|███| 34/34 [00:00<00:00, 5591.53it/s, # trackers=1]
2019-05-03 

2019-05-03 21:21:13 INFO     rasa_core.agent  - Persisted model to '/work/bot/models/dialogue'
make: Leaving directory '/work/bot'


* `rasa_core.evaluate` é o método utilizado para verificar a qualidade das suas `stories`.

In [5]:
!python -m rasa_core.evaluate --core $BOT_MODELS_DIALOGUE_PATH -s $BOT_STORIES_PATH -o results

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Calling `rasa_core.evaluate` is deprecated. Please use `rasa_core.test` instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

2019-05-03 21:21:17.252292: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-05-03 21:21:17.270392: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2905000000 Hz
2019-05-03 21:21:17.271557: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55efe976d490 executing computations on platform Host. Devices:
2019-05-03 21:21:17.271656: I tensorflow/compile

* Caso algum problme seja encotrado ele será descrito no arquivo `failed_stories.md`

In [6]:
!cat results/failed_stories.md

<!-- All stories passed -->

* Se tudo estiver correto você deverá ver a mensagem:

`<!-- All stories passed -->`

* Outro arquivo gerado é  o `story_confmat.pdf` uma matriz de confuzão onde é possível visualizar a relação entre as `utters` (mensagens enviadas .

In [7]:
from IPython.display import IFrame
IFrame("results/story_confmat.pdf", width=900, height=900)

## Referências:

O Rasa está em constante evolução, alguns links úteis para a construção deste jupyter-notebook e para a análise das `stories` são:

* [Evaluation](https://rasa.com/docs/core/evaluation/)
* [Debugging](https://rasa.com/docs/core/debugging/)